# Reading and Writing Excel Files

# Using pandas with Excel Files

## Case Study: Excel Reporting

In [ ]:
import pandas as pd
df = pd.read_excel('sales_data/new/January.xlsx')
df.info()

## Reading Excel Files with pandas

In [ ]:
df1 = pd.read_excel('xl/stores.xlsx', sheet_name='2019', skiprows=1,
                    usecols=['Store', 'Employees'],
                    index_col='Store', nrows=2)
df1

In [ ]:
sheets = pd.read_excel('xl/stores.xlsx', sheet_name=['2019', '2020'],
                       skiprows=1, usecols='B:C')
sheets['2019'].head(2)

In [ ]:
df1 = pd.read_excel('xl/stores.xlsx', sheet_name=0,
                    skiprows=2, usecols='B:C,F', skipfooter=3,
                    header=None,
                    names=['Branch', 'Employee_Count', 'Is_Flagship'])
df1

In [ ]:
df1 = pd.read_excel('xl/stores.xlsx', sheet_name='2019',
                    skiprows=1, usecols='B,C,F', skipfooter=2,
                    na_values='MISSING', keep_default_na=False)
df1

In [ ]:
with pd.ExcelFile('xl/stores.xls') as f:
    df1 = pd.read_excel(f, '2019', skiprows=1, usecols='B:F', nrows=2)
    df2 = pd.read_excel(f, '2020', skiprows=1, usecols='B:F', nrows=2)

df1

In [ ]:
stores = pd.ExcelFile('xl/stores.xlsx')
stores.sheet_names

In [ ]:
url = ('https://github.com/fzumstein/python-for-excel/'
      'blob/master/ch05/xl/stores.xlsx?raw=true')
pd.read_excel(url, skiprows=1, usecols='B:E', nrows=2)

## Writing Excel Files with pandas

In [ ]:
import numpy as np
import datetime as dt

data=[[dt.datetime(2020,1,1, 10, 13), 2.222, 1, True],
      [dt.datetime(2020,1,2), np.nan, 2, False],
      [dt.datetime(2020,1,2), np.inf, 3, True]]
df = pd.DataFrame(data=data,
                  columns=['A', 'B', 'C', 'D'])
df.index.name='index'
df

In [ ]:
df.to_excel('written_with_pandas.xlsx', sheet_name='Output',
            startrow=1, startcol=1, index=True, header=True,
            na_rep='<NA>', inf_rep='<INF>')

In [ ]:
with pd.ExcelWriter('written_with_pandas2.xlsx') as writer:
    df.to_excel(writer, sheet_name='Sheet1', startrow=1, startcol=1)
    df.to_excel(writer, sheet_name='Sheet1', startrow=10, startcol=1)
    df.to_excel(writer, sheet_name='Sheet2')

# Reader and Writer Packages
## OpenPyXL
### Reading with OpenPyXL

In [ ]:
import openpyxl
import excel

In [ ]:
# Open the workbook to read cell values
# The file is automatically closed again after loading the data
book = openpyxl.load_workbook('xl/stores.xlsx', data_only=True)

In [ ]:
# Get a worksheet object by name or index (0-based)
sheet = book['2019']
sheet = book.worksheets[0]

In [ ]:
# Get a list with all sheet names
book.sheetnames

In [ ]:
# Loop through the sheet objects
# Instead of 'name', openpyxl uses 'title'
for i in book.worksheets:
    print(i.title)

In [ ]:
# Getting the dimensions,
# i.e. the used range of the sheet
sheet.max_row, sheet.max_column

In [ ]:
# Read the value of a single cell
# using 'A1' notation and using cell indices (1-based)
sheet['B6'].value
sheet.cell(row=6, column=2).value

In [ ]:
# Read in a range of cell values by using our excel module
data = excel.read(book['2019'], (2, 2), (8, 6))
data[:2]

### Writing with OpenPyXL

In [ ]:
import openpyxl
from openpyxl.drawing.image import Image
from openpyxl.styles import Font, colors
from openpyxl.chart import BarChart, Reference
import excel

# Instantiate a workbook
book = openpyxl.Workbook()

# Get the first sheet and give it name
sheet = book.active
sheet.title = 'Sheet1'

# Writing individual cells using A1 notation
# and cell indices (1-based)
sheet['A1'].value = 'Hello 1'
sheet.cell(row=2, column=1, value='Hello 2')

# Font formatting
font_format = Font(color='FF0000', bold=True)
sheet['A3'].value = 'Hello 3'
sheet['A3'].font = font_format

# Number formatting
sheet['A4'].value = 3.3333
sheet['A4'].number_format = '0.00'

# Date formatting
sheet['A5'].value = dt.date(2016, 10, 13)
sheet['A5'].number_format = 'MM/DD/YY'

# Formula
sheet['A6'].value = '=A4 * 2'

# Image
sheet.add_image(Image('python.png'), 'C1')

# 2-dimensional list
data = [[None, 'Last Year', 'This Year'],
        ['North', 2, 3],
        ['South', 5, 6]]
excel.write(sheet, data, 'A10')

# Chart
chart = BarChart()
chart.type = 'col'
chart.title = 'Sales Per Region'
chart.x_axis.title = 'Regions'
chart.y_axis.title = 'Sales'
data = Reference(sheet, min_col=2, min_row=10,
                 max_row=12, max_col=3)
categories = Reference(sheet, min_row=11, min_col=1,
                       max_row=12, max_col=1)
chart.add_data(data, titles_from_data=True)
chart.set_categories(categories)
sheet.add_chart(chart, "A15")

# Saving the workbook creates the physical file
book.save('openpyxl.xlsx')

In [ ]:
book = openpyxl.Workbook()
book.template = True
book.save('template.xltx')

### Editing with OpenPyXL

In [ ]:
# Read the stores.xlsx file, change a cell
# and store it under a new location/name
book = openpyxl.load_workbook('xl/stores.xlsx')
book['2019']['A1'].value = 'modified'
book.save('stores_edited.xlsx')

In [ ]:
book = openpyxl.load_workbook('xl/macro.xlsm', keep_vba=True)
book['Sheet1']['A1'].value = 'Click the button!'
book.save('macro_openpyxl.xlsm')

## XlsxWriter

In [ ]:
import datetime as dt
import xlsxwriter
import excel

# Instantiate a workbook
book = xlsxwriter.Workbook('xlxswriter.xlsx')

# Add a sheet and give it a name
sheet = book.add_worksheet('Sheet1')

# Writing individual cells using A1 notation
# and cell indices (0-based)
sheet.write('A1', 'Hello 1')
sheet.write(1, 0, 'Hello 2')

# Font formatting
font_format = book.add_format({'font_color': '#FF0000',
                               'bold': True})
sheet.write('A3', 'Hello 3', font_format)

# Number formatting
number_format = book.add_format({'num_format': '0.00'})
sheet.write('A4', 3.3333, number_format)

# Date formatting
date_format = book.add_format({'num_format': 'mm/dd/yy'})
sheet.write('A5', dt.date(2016, 10, 13), date_format)

# Formula
sheet.write('A6', '=A4 * 2')

# Image
sheet.insert_image(0, 2, 'python.png')

# 2-dimensional list
data = [[None, 'old', 'new'],
        ['one', 2, 3],
        ['two', 5, 6]]
excel.write(sheet, data, 'A10')

# Chart
chart = book.add_chart({'type': 'column'})
chart.set_title({'name': 'Sales per Region'})
chart.add_series({'name': '=Sheet1!B10',
                  'categories': '=Sheet1!A11:A12',
                  'values': '=Sheet1!B11:B12'})
chart.add_series({'name': '=Sheet1!C10',
                  'categories': '=Sheet1!A11:A12',
                  'values': '=Sheet1!C11:C12'})
chart.set_x_axis({'name': 'Regions'})
chart.set_y_axis({'name': 'Sales'})
sheet.insert_chart('A15', chart)

# Closing the workbook creates the physical file
book.close()

In [ ]:
book = xlsxwriter.Workbook('macro_xlxswriter.xlsm')
sheet = book.add_worksheet('Sheet1')
sheet.write('A1', 'Click the button!')
book.add_vba_project('xl/vbaProject.bin')
sheet.insert_button('A3', {'macro':   'Hello', 'caption': 'Button 1',
                           'width':   130, 'height':  35})
book.close()

## pyxlsb

In [ ]:
import pyxlsb
import excel

# Loop through sheets. With pyxlsb, the workbook
# and sheets can be used as context managers
# book.sheets returns a list of sheet names, not objects
with pyxlsb.open_workbook('xl/stores.xlsb') as book:
    for sheet_name in book.sheets:
        with book.get_sheet(sheet_name) as sheet:
            dim = sheet.dimension
            print(f'Sheet "{sheet_name}" has ' 
                  f'{dim.h} rows and {dim.w} cols')

In [ ]:
# Read in the values of a range of cells by using our excel module
# Instead of '2019', you could also use its index (1-based)
with pyxlsb.open_workbook('xl/stores.xlsb') as book:
    with book.get_sheet('2019') as sheet:
        data = excel.read(sheet, 'D2', 'E3')
data

In [ ]:
from pyxlsb import convert_date
convert_date(data[1][1])

In [ ]:
df = pd.read_excel('xl/stores.xlsb', engine='pyxlsb')

## xlrd, xlwt and xlutils

### Reading with xlrd

In [ ]:
import xlrd
import xlwt
from xlwt.Utils import cell_to_rowcol2
import xlutils
import excel

In [ ]:
# Open the workbook to read cell values. The file is
# automatically closed again after loading the data.
book = xlrd.open_workbook('xl/stores.xls')

In [ ]:
# Get a list with all sheet names
book.sheet_names()

In [ ]:
# Loop through the sheet objects
for sheet in book.sheets():
    print(sheet.name)

In [ ]:
# Get a sheet object by name or index (0-based)
sheet = book.sheet_by_index(0)
sheet = book.sheet_by_name('2019')

In [ ]:
# Dimensions
sheet.nrows, sheet.ncols

In [ ]:
# Read the value of a single cell
# using 'A1' notation and using cell indices (0-based)
# The * unpacks the returned tuple into individual arguments
sheet.cell(*cell_to_rowcol2('B3')).value
sheet.cell(2, 1).value

In [ ]:
# Read in a range of cell values by using our excel module
excel.read(sheet, 'B2', 'C3')

### Writing with xlwt

In [ ]:
import xlwt
from xlwt.Utils import cell_to_rowcol2
import datetime as dt
import excel

# Instantiate a workbook
book = xlwt.Workbook()

# Add a sheet and give it a name
sheet = book.add_sheet('Sheet1')

# Writing individual cells using A1 notation
# and cell indices (0-based)
# The * unpacks the returned tuple into individual arguments
sheet.write(*cell_to_rowcol2('A1'), 'Hello 1')
sheet.write(r=1, c=0, label='Hello 2')

# Font formatting
font_format = xlwt.easyxf('font: bold 1, color red')
sheet.write(r=2, c=0, label='Hello 3', style=font_format)

# Number formatting
number_format = xlwt.easyxf(num_format_str='0.00')
sheet.write(3, 0, 3.3333, number_format)

# Date formatting
date_format = xlwt.easyxf(num_format_str='MM/DD/YYYY')
sheet.write(4, 0, dt.datetime(2012, 2, 3), date_format)

# Formula
sheet.write(5, 0, xlwt.Formula('A3'))

# 2-dimensional list
data = [[None, 'old', 'new'],
        ['one', 2, 3],
        ['two', 5, 6]]
excel.write(sheet, data, 'A10')

# Picture (only allows to add bmp format)
sheet.insert_bitmap('python.bmp', 0, 2)

# This writes the file to disk
book.save('xlwt.xls')

### Editing with xlutils

In [ ]:
import xlutils.copy

book = xlrd.open_workbook('xl/stores.xls', formatting_info=True)
book = xlutils.copy.copy(book)
book.get_sheet(0).write(0, 0, 'changed!')
book.save('stores_edited.xls')

# Advanced Topics
## Working with Big Files

### Writing with OpenPyXL

In [ ]:
book = openpyxl.Workbook(write_only=True)
sheet = book.create_sheet()
for _ in range(1000):
    sheet.append(list(range(200)))
book.save('openpyxl_optimized.xlsx')

### Writing with XlsxWriter

In [ ]:
book = xlsxwriter.Workbook('xlsxwriter_optimized.xlsx',
                           options={'constant_memory': True})
sheet = book.add_worksheet()
for r in range(1000):
    sheet.write_row(r , 0, list(range(200)))
book.close()

### Reading with xlrd

In [ ]:
with xlrd.open_workbook('xl/stores.xls', on_demand=True) as book:
    sheet = book.sheet_by_index(0)  # only loads the first sheet

In [ ]:
with xlrd.open_workbook('xl/stores.xls', on_demand=True) as book:
    with pd.ExcelFile(book, engine='xlrd') as f:
        df = pd.read_excel(f, sheet_name=0)

### Reading with OpenPyXL

In [ ]:
book = openpyxl.load_workbook('xl/big.xlsx',
                              data_only=True, read_only=True,
                              keep_links=False)
sheet = book.worksheets[0]
book.close()  # required with read_only=True

### Reading in Parallel

In [ ]:
%%time
data = pd.read_excel('xl/big.xlsx', sheet_name=None, engine='openpyxl')

In [ ]:
%%time
import pandas_parallel
data = pandas_parallel.read_excel('xl/big.xlsx', sheet_name=None)

## Formatting DataFrames in Excel

In [ ]:
with pd.ExcelFile('xl/stores.xlsx', engine='openpyxl') as xlfile:
    df = pd.read_excel(xlfile, sheet_name='2020')

    # Get the OpenPyXL workbook object
    book = xlfile.book

    # from here on, it's OpenPyXL code
    sheet = book['2019']
    value = sheet['B3'].value

In [ ]:
with pd.ExcelWriter('pandas_and_openpyxl.xlsx',
                    engine='openpyxl') as writer:
    df = pd.DataFrame({'col1': [1, 2, 3, 4], 'col2': [5, 6, 7, 8]})
    df.to_excel(writer, 'Sheet1', startrow=4, startcol=2)

    # Get the OpenPyXL workbook and sheet objects
    book = writer.book
    sheet = writer.sheets['Sheet1']

    # from here on, it's OpenPyXL code
    sheet['A1'].value = 'Hello'

In [ ]:
df = pd.DataFrame({'col1': [1, -2], 'col2': [-3, 4]},
                   index=['row1', 'row2'])
df.index.name = 'ix'
df

In [ ]:
# Formatting index/headers with OpenPyXL
from openpyxl.styles import PatternFill
with pd.ExcelWriter('formatting_openpyxl.xlsx',
                    engine='openpyxl') as writer:
    # Default formatting
    df.to_excel(writer, startrow=0, startcol=0)
    # Custom formatting    
    startrow, startcol = 0, 5
    df.to_excel(writer, header=False, index=False,
                startrow=startrow + 1, startcol=startcol + 1)
    sheet = writer.sheets['Sheet1']
    style = PatternFill(fgColor="D9D9D9", fill_type="solid")
    # header
    for i, col in enumerate(df.columns):
        sheet.cell(row=startrow + 1, column=i + startcol + 2,
                   value=col).fill = style
    # index
    for i, row in enumerate([df.index.name if df.index.name else None]
                            + list(df.index)):
        sheet.cell(row=i + startrow + 1, column=startcol + 1,
                   value=row).fill = style

In [ ]:
# Formatting index/headers with XlsxWriter
with pd.ExcelWriter('formatting_xlsxwriter.xlsx',
                    engine='xlsxwriter') as writer:
    # Default formatting
    df.to_excel(writer, startrow=0, startcol=0)
    # Custom formatting
    startrow, startcol = 0, 5
    df.to_excel(writer, header=False, index=False,
                startrow=startrow + 1, startcol=startcol + 1)
    book = writer.book
    sheet = writer.sheets['Sheet1']
    style = book.add_format({'bg_color': '#D9D9D9'})
    # header
    for i, col in enumerate(df.columns):
        sheet.write(startrow, startcol + i + 1, col, style)
    # index
    for i, row in enumerate([df.index.name if df.index.name else None]
                            + list(df.index)):
        sheet.write(startrow + i, startcol, row, style)

In [ ]:
# Formatting the data part with OpenPyXL
from openpyxl.styles import Alignment
with pd.ExcelWriter('data_format_openpyxl.xlsx',
                    engine='openpyxl') as writer:
    df.to_excel(writer)
    book = writer.book
    sheet = writer.sheets['Sheet1']
    nrows, ncols = df.shape
    for row in range(nrows):
        for col in range(ncols):
            cell = sheet.cell(row=row + 2,
                       column=col + 2)
            cell.number_format = '0.000'
            cell.alignment = Alignment(horizontal='center')

In [ ]:
# Formatting the data part with XlsxWriter
with pd.ExcelWriter('data_format_xlsxwriter.xlsx',
                    engine='xlsxwriter') as writer:
    df.to_excel(writer)
    book = writer.book
    sheet = writer.sheets['Sheet1']
    number_format = book.add_format({'num_format': '0.000',
                                     'align': 'center'})
    sheet.set_column(first_col=1, last_col=2,
                     cell_format=number_format)

In [ ]:
df.style.applymap(lambda x: 'number-format: 0.000;'
                            'text-align: center')\
        .to_excel('styled.xlsx')

In [ ]:
df = pd.DataFrame({'A': [dt.date(2020, 1, 1)],
                   'B': [dt.datetime(2020, 1, 1, 10)]})
with pd.ExcelWriter('date.xlsx',
                    date_format='YYYY-MM-DD',
                    datetime_format='YYYY-MM-DD HH:MM:SS') as writer:
    df.to_excel(writer)